# Installing Packages / Importing Libraries

In [ ]:
pip install missingno

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

# Importing Data

In [ ]:
df1 = pd.read_csv('May.csv', delimiter=';', encoding='cp1252' , header=0, skiprows=[1] , index_col=[0])
df2 = pd.read_csv('June.csv', delimiter=';', encoding='cp1252' , header=0, skiprows=[1] , index_col=[0])
df3 = pd.read_csv('July.csv', delimiter=';', encoding='cp1252', header=0, skiprows=[1] , index_col=[0])
df4 = pd.read_csv('August.csv', delimiter=';', encoding='cp1252', header=0, skiprows=[1] , index_col=[0])
df5 = pd.read_csv('September.csv', delimiter=';', encoding='cp1252', header=0, skiprows=[1] , index_col=[0])
df6 = pd.read_csv('October.csv', delimiter=';', encoding='cp1252', header=0, skiprows=[1] , index_col=[0])

**July's dataframe has 131 features in contrast to the other dataframes which have 95 to 99 features, some of them with almost identical labels but different values.**

In [ ]:
df3

The different columns that July's dataframe has.

In [ ]:
diff_col = []
for col in df3.columns:
  if col not in df1.columns:
    diff_col = diff_col + [col]
diff_col

Exploring columns with similar named features.

In [ ]:
df3[df3['mill_in_pres_mbar_pv'] != df3['mill_in_press_mbar_pv']][['mill_in_pres_mbar_pv','mill_in_press_mbar_pv']].sample(20)

Concatenating the dataframes and dropping id column.

In [ ]:
dataframes = [df1, df2, df3, df4, df5, df6]

# Concatenate the DataFrames along the rows (axis=0)
df = pd.concat(dataframes, axis=0, ignore_index=True)

df = df.drop(columns='_id')

In [ ]:
df.replace({'ERROR' : np.NaN}, inplace=True)

In [ ]:
df

# Mill Operation

In [ ]:
df['mill_motor_pwr_kw_pv'].value_counts()

In [ ]:
df['mill_motor_pwr_kw_pv'].dtype

In [ ]:
df['mill_motor_pwr_kw_pv'] = df['mill_motor_pwr_kw_pv'].astype(float)

In [ ]:
sns.distplot(df['mill_motor_pwr_kw_pv'])

plt.xlabel('Mill Motor (kW)')

In [ ]:
#number of missing values
df['mill_motor_pwr_kw_pv'].isna().sum()

In [ ]:
#changing random missing values during normal mill operation
df = df.interpolate(limit=2, limit_direction="forward")

In [ ]:
#continuously missing values
df['mill_motor_pwr_kw_pv'].isna().sum()

In [ ]:
msno.matrix(df[['mill_motor_pwr_kw_pv']],figsize=(2,5))

In [ ]:
df['mill_operation'].isna().sum()

In [ ]:
df['mill_operation'].interpolate(limit=2, limit_direction='forward', method='ffill', inplace=True)

In [ ]:
df['mill_operation'].isna().sum() # same as mill motor missing values

In [ ]:
df.dropna(subset=['mill_operation','mill_motor_pwr_kw_pv'], inplace=True)

In [ ]:
#drop missing values for mill operation and mill motor
df['mill_operation'].isna().sum() , df['mill_motor_pwr_kw_pv'].isna().sum()

In [ ]:
df['mill_operation'] = df['mill_operation'].astype(float)

**Replacing zeros with NaN in order to interpolate and visualize off periods.**

In [ ]:
#replacing zeros with NaN in order to use interpolate
df['mill_operation'].replace({0.0 : np.NaN}, inplace=True)

In [ ]:
#replacing random zeros in 1 minute intervals
df['mill_operation'].interpolate(limit=2, limit_direction='forward', method='ffill', inplace=True)

In [ ]:
df['mill_operation'].isna().sum()

In [ ]:
msno.matrix(df[['mill_operation']],figsize=(2,5))

In [ ]:
#distribution of mill motor after deleting mill operation = 0 values
sns.distplot(df[df['mill_operation']==1]['mill_motor_pwr_kw_pv'])

plt.xlabel('Mill Motor (kW)')

In [ ]:
df = df[df['mill_operation']==1]
df.reset_index(drop=True,inplace=True)

**The length of dataframe after deleting the off period measurements is 349883 rows**

In [ ]:
len(df)

In [ ]:
# df = df[(df['mill_motor_pwr_kw_pv'] > 250) & (df['mill_motor_pwr_kw_pv'] < 1750)]
# df = df.reset_index(drop=True)

# sns.distplot(df['mill_motor_pwr_kw_pv'])

In [ ]:
(df['mill_motor_pwr_kw_pv'].dtype , df['mill_motor_pwr_kw_pv'].isna().sum())

# Total Feed missing values handling

In [ ]:
df[['total_feed_ton/h_pv','total_feed_ton_h_pv']]

In [ ]:
# missing values of two columns
df['total_feed_ton/h_pv'].isna().sum() , df['total_feed_ton_h_pv'].isna().sum()

In [ ]:
df['total_feed_ton/h_pv'].replace({'ERROR' : np.NaN}, inplace=True)

In [ ]:
#filling missing values from duplicated column
df['total_feed_ton_h_pv'] = df['total_feed_ton_h_pv'].fillna(df['total_feed_ton/h_pv'])

In [ ]:
df['total_feed_ton_h_pv'].isna().sum()

In [ ]:
df = df.drop(columns=['total_feed_ton/h_pv'])

In [ ]:
df['total_feed_ton_h_pv'].interpolate(method='ffill', inplace=True)

In [ ]:
df['total_feed_ton_h_pv'].isna().sum()

In [ ]:
df['total_feed_ton_h_pv'] = df['total_feed_ton_h_pv'].astype(float)

In [ ]:
plt.figure(figsize=(8,6))

sns.distplot(df[df['cement_type_cpii_bool_pv']==1]['total_feed_ton_h_pv'],label='CPII')

sns.distplot(df[df['cement_type_cpiv_bool_pv']==1]['total_feed_ton_h_pv'],label='CPIV')

plt.legend()

**It seems that there is a difference in total feeding supply for the two cement types.**

# Datetime Format

In [ ]:
# Import the 're' module for working with regular expressions
import re

# Retrieve the 'date' values from 'august' DataFrame where the value does not start with '2019'
date1 = df[~df['date'].str.startswith('2019')]['date']

# Retrieve the 'date' values from 'august' DataFrame where the value starts with '2019'
date2 = df[df['date'].str.startswith('2019')]['date']

# Randomly sample 20 values from 'date1'
date1.sample(20)

In [ ]:
# Modify the 'date1' values by adding a leading '0' to the hour component if it's missing
date1 = date1.apply(lambda s: re.sub(r'(\d{1,2}\/\d{1}\/\d{4})\s(\d{1}:\d{2})', r'\1 0\2', s))

# Add a leading '0' to the 'date1' values if they match a specific date and time format
date1 = date1.apply(lambda s: '0' + s[0:] if re.match('\d{1}\/\d{1}\/\d{4}\s\d{2}:\d{2}', s) else s)

# Modify the 'date1' values by adding a leading '0' to the day component if it's missing
date1 = date1.apply(lambda s: s[0:3] + '0' + s[3:] if re.match('\d{2}\/\d{1}\/\d{4}\s\d{2}:\d{2}', s) else s)

# Convert the modified 'date1' values to datetime format with the specified format '%d/%m/%Y %H:%M'
date1 = pd.to_datetime(date1, format='%d/%m/%Y %H:%M')

In [ ]:
date2 = pd.to_datetime(date2,format='%Y-%m-%d %H:%M:%S.%f')
date2

In [ ]:
df['date'] = pd.concat([date1,date2])
df['date']

**Datetime formatted and sorted.**

In [ ]:
df['date'].is_monotonic_increasing

In [ ]:
df['index'] = df.index[:]

In [ ]:
df = df.sort_values(['date','index'])

In [ ]:
df['date'].is_monotonic_increasing

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.drop(columns='index',inplace=True)

In [ ]:
df['date'] = df['date'].round('s')

In [ ]:
df['date']

In [ ]:
from datetime import timedelta

for i in range(len(df)-1):
  if df.loc[i+1,'date'] == df.loc[i,'date']:
    df.loc[i+1,'date'] = df.loc[i+1,'date'] + timedelta(seconds=30)

In [ ]:
df['date'].duplicated().sum()

In [ ]:
df['index'] = df.index[:]
df = df.sort_values(['date','index'])

In [ ]:
df.reset_index(drop=True,inplace=True)
df.drop(columns='index',inplace=True)

# Handling Separator Motor ERRORS and Missing Values

In [ ]:
df['separator_pwr_kw_pv'].dtype

In [ ]:
#missing values
df['separator_pwr_kw_pv'].isna().sum()

In [ ]:
#errors
(df['separator_pwr_kw_pv'] == 'ERROR').sum()

In [ ]:
df['separator_pwr_kw_pv'] = df['separator_pwr_kw_pv'].replace({'ERROR' : np.NaN})

In [ ]:
df['separator_pwr_kw_pv'] = df['separator_pwr_kw_pv'].astype(float)

In [ ]:
df['separator_pwr_kw_pv'] = df['separator_pwr_kw_pv'].interpolate()

In [ ]:
sns.distplot(df['separator_pwr_kw_pv'])

In [ ]:
plt.figure(figsize = (20,8))

plt.scatter(df['date'],df['separator_pwr_kw_pv'])
plt.xlabel('Month')
plt.ylabel('Separator Motor KW')

In [ ]:
df[df['separator_pwr_kw_pv'] < 40]['separator_pwr_kw_pv'].count()

In [ ]:
df['separator_pwr_kw_pv'] = df['separator_pwr_kw_pv'].apply(lambda x: np.NaN if x < 40 or x > 140 else x)


In [ ]:
df['separator_pwr_kw_pv'] = df['separator_pwr_kw_pv'].interpolate(method='linear')

In [ ]:
plt.figure(figsize = (20,8))

plt.scatter(df['date'],df['separator_pwr_kw_pv'],c='blue')
plt.xlabel('Month')
plt.ylabel('Separator Motor KW')

In [ ]:
(df['separator_pwr_kw_pv'].dtypes , df['separator_pwr_kw_pv'].isna().sum())

# Separator Speed , Mill DP Handling Missing Values and ERRORS

**Separator Speed**

In [ ]:
df['separator_speed_rpm_pv'].dtype

In [ ]:
(df['separator_speed_rpm_pv'] == 'ERROR').sum() # same ERRORS as separator motor

In [ ]:
df['separator_speed_rpm_pv'].replace({'ERROR' : np.NaN},inplace=True)

In [ ]:
df['separator_speed_rpm_pv'] = df['separator_speed_rpm_pv'].astype(float)

In [ ]:
df['separator_speed_rpm_pv'].isna().sum()

In [ ]:
df['separator_speed_rpm_pv'] = df['separator_speed_rpm_pv'].interpolate()

In [ ]:
sns.distplot(df['separator_speed_rpm_pv'])

In [ ]:
(df['separator_speed_rpm_pv'].dtypes , df['separator_speed_rpm_pv'].isna().sum())

In [ ]:
len(df)

In [ ]:
sns.violinplot(df['separator_speed_rpm_pv'])

**Mill DP**

In [ ]:
df['mill_dp_mbar_pv'].dtype

In [ ]:
df['mill_dp_mbar_pv'].isna().sum() 

In [ ]:
(df['mill_dp_mbar_pv'] == 'ERROR').sum() # same as separator motor and speed

In [ ]:
df['mill_dp_mbar_pv'].replace({'ERROR' : np.NaN},inplace=True)

In [ ]:
msno.matrix(df[['mill_dp_mbar_pv']], figsize=(2,5))

In [ ]:
df['mill_dp_mbar_pv'] = df['mill_dp_mbar_pv'].astype(float)

In [ ]:
df['mill_dp_mbar_pv'] = df['mill_dp_mbar_pv'].interpolate()

In [ ]:
sns.distplot(df['mill_dp_mbar_pv'])

In [ ]:
(df['mill_dp_mbar_pv'].dtypes , df['mill_dp_mbar_pv'].isna().sum())

#  Separator Motor KW for different cement types.

In [ ]:
plt.figure(figsize=(20,8))

plt.scatter(df[df['cement_type_cpii_bool_pv'] == 1]['date'],df[df['cement_type_cpii_bool_pv'] == 1]['separator_pwr_kw_pv'])
plt.scatter(df[df['cement_type_cpii_bool_pv'] == 0]['date'],df[df['cement_type_cpii_bool_pv'] == 0]['separator_pwr_kw_pv'])

# Exploring Distribution of certain features.

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
sns.distplot(df[df['separator_speed_rpm_pv'] < 600]['separator_pwr_kw_pv'])
plt.title('Separator Motor Distribution for Separator Speed < 600 rpm')

plt.subplot(122)
sns.distplot(df[df['separator_speed_rpm_pv'] > 600]['separator_pwr_kw_pv'])
plt.title('Separator Motor Distribution for Separator Speed > 600 rpm')

plt.suptitle('Separator Motor Distr splitted by Separator Speed')

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
sns.distplot(df[df['separator_speed_rpm_pv'] < 600]['mill_motor_pwr_kw_pv'])
plt.title('Mill Motor Distribution for Separator Speed < 600 rpm')

plt.subplot(122)
sns.distplot(df[df['separator_speed_rpm_pv'] > 600]['mill_motor_pwr_kw_pv'])
plt.title('Mill Motor Distribution for Separator Speed > 600 rpm')

plt.suptitle('Mill Motor Distr splitted by Separator Speed')

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
sns.distplot(df[df['cement_type_cpii_bool_pv'] == 1]['separator_pwr_kw_pv'])
plt.title('Separator Motor Distribution for CPII')
plt.xlabel('Separator Motor (kW)')

plt.subplot(122)
sns.distplot(df[df['cement_type_cpii_bool_pv'] == 0]['separator_pwr_kw_pv'])
plt.title('Separator Motor Distribution for CPIV')
plt.xlabel('Separator Motor (kW)')

**Identically distributed separator and mill motor for different cement types**

**Possible Time Delay**

In [ ]:
# from scipy import signal
# x = df['separator_speed_rpm_pv']
# y = df['separator_pwr_kw_pv']
# correlation = signal.correlate(x-np.mean(x), y - np.mean(y), mode="full")
# lags = signal.correlation_lags(len(x), len(y), mode="full")
# lag = lags[np.argmax(abs(correlation))]
# lag

# Smoothing TimeSeries of Mill Motor

In [ ]:
plt.figure(figsize=(20,8))

fig, ax1 = plt.subplots(figsize=(20,8))

ax1.plot(df[df['date'] > '2019-10-29']['date'],df[df['date'] > '2019-10-29']['mill_motor_pwr_kw_pv'])


ax2=ax1.twinx()

ax2.plot(df[df['date'] > '2019-10-29']['date'],df[df['date'] > '2019-10-29']['separator_speed_rpm_pv'],color='red')

In [ ]:
plt.plot(df['separator_speed_rpm_pv'])

In [ ]:
sns.distplot(df[(df['separator_speed_rpm_pv']>500) & (df['separator_speed_rpm_pv']<800)]['mill_motor_pwr_kw_pv'])

# Stopping periods of Mill

In [ ]:
from pandas.core.arrays.timedeltas import Timedelta
from pandas.core.groupby.groupby import Timestamp

In [ ]:
df.loc[0,'stopping_period_interval'] = Timedelta(0)

In [ ]:
for i in range(len(df)-1):
  df.loc[i+1,'stopping_period_interval'] = df.loc[i+1,'date'] - df.loc[i,'date']

In [ ]:
df['stopping_period_interval'].sort_values(ascending=False).head(300)

In [ ]:
df

# Cement Type Changing

In [ ]:
df['cement_type_cpii_bool_pv'].value_counts()

In [ ]:
df['cement_type_cpii_bool_pv'].replace({'ERROR' : np.NaN}, inplace=True)

In [ ]:
df['cement_type_cpii_bool_pv'].isna().sum()

In [ ]:
df = df.dropna(subset=['cement_type_cpii_bool_pv'])
df.reset_index(inplace=True,drop=True)

In [ ]:
df['cement_type_cpii_bool_pv'] = df['cement_type_cpii_bool_pv'].astype(float)

In [ ]:
df['cement_type_cpii_bool_pv'].isna().sum()

In [ ]:
df['cement_type_cpii_bool_pv'].value_counts()

In [ ]:
# cement type change boolean 
df.loc[0,'change_cement_type_boolean'] = 0
for i in range(len(df)-1):
  df.loc[i+1,'change_cement_type_boolean'] = abs(df.loc[i+1,'cement_type_cpii_bool_pv'] - df.loc[i,'cement_type_cpii_bool_pv'])

In [ ]:
df['change_cement_type_boolean'].value_counts()

**We have 179 changes of cement type during 6 month milling process**

# Stopping periods for cement type change

It seems that cement type change in the mill can be performed instantly! 

In [ ]:
df[df['change_cement_type_boolean'] == 1]['stopping_period_interval'].sort_values(ascending=False)

In [ ]:
df[df['change_cement_type_boolean'] == 1]['stopping_period_interval'].value_counts()

# Starting periods of mill

In [ ]:
df['stopping_period_interval'].sort_values(ascending=False)[117268]

In [ ]:
plt.plot(df.loc[117268:117330]['mill_motor_pwr_kw_pv'])
plt.plot(df.loc[117268:117330]['separator_speed_rpm_pv'])

In [ ]:
plt.plot(df.loc[117268:117330]['total_feed_ton_h_pv'])

**Trying to define normal operation conditions with respect to total feed and separator speed short-term stability.**

In [ ]:
df['stopping_period_interval'].sort_values(ascending=False)[56824]

In [ ]:
plt.plot(df.loc[56824:57000]['mill_motor_pwr_kw_pv'])
plt.plot(df.loc[56824:57000]['separator_speed_rpm_pv'])

In [ ]:
plt.plot(df.loc[56824:57000]['total_feed_ton_h_pv'])

In [ ]:
dfc = df.copy()

In [ ]:
dfc.loc[56824:57000]['total_feed_ton_h_pv'].rolling(10).mean()

In [ ]:
dfc.loc[56824:57000]['total_feed_ton_h_pv']

In [ ]:
dfc.loc[56824:57000]['total_feed_ton_h_pv'].rolling(10).std()

In [ ]:
dfc['total_feed_rolling_avg_ton_h'] = dfc['total_feed_ton_h_pv'].rolling(10).mean()

In [ ]:
dfc['total_feed_rolling_std_ton_h'] = dfc['total_feed_ton_h_pv'].rolling(10).std()

In [ ]:
dfc['separator_speed_rolling_std_rpm'] = dfc['separator_speed_rpm_pv'].rolling(10).std()

In [ ]:
plt.plot(dfc[(dfc['total_feed_rolling_std_ton_h'] < 4) & (dfc['separator_speed_rolling_std_rpm'] < 4) & (dfc['date'] > '2019-09-10') & (dfc['date'] < '2019-09-11')]['total_feed_ton_h_pv'])

In [ ]:
sns.distplot(dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 3)]['mill_motor_pwr_kw_pv'])

plt.xlabel('Mill Motor (kW)')

In [ ]:
plt.figure(figsize=(15,8))

x = dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 3)]['date']
y = dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 3)]['mill_motor_pwr_kw_pv']

plt.scatter(x,y)

In [ ]:
dfc[dfc['date'] > '2019-07-26']['mill_motor_pwr_kw_pv'].mean()

In [ ]:
dfc[dfc['date'] < '2019-07-26']['mill_motor_pwr_kw_pv'].mean()

In [ ]:
plt.figure(figsize=(15,8))

y = dfc[(dfc['mill_motor_pwr_kw_pv'] > 750) & (dfc['mill_motor_pwr_kw_pv'] < 1500)]['mill_motor_pwr_kw_pv']
x = dfc[(dfc['mill_motor_pwr_kw_pv'] > 750) & (dfc['mill_motor_pwr_kw_pv'] < 1500)]['date']

plt.scatter(x,y)

In [ ]:
plt.figure(figsize=(20,8))

fig, ax1 = plt.subplots(figsize=(20,8))

ax1.plot(dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 3) & (dfc['date'] > '2019-09-01') & (dfc['date'] < '2019-09-10')]['separator_pwr_kw_pv'])

ax2 = ax1.twinx()

ax2.plot(dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 3) & (dfc['date'] > '2019-09-01') & (dfc['date'] < '2019-09-10')]['separator_speed_rpm_pv'],color='red')

**Keeping at the original dataframe values with std < 4 around the mean value of 5 minutes ahead.**

In [ ]:
dfc['total_feed_rolling_avg_ton_h'] = dfc['total_feed_ton_h_pv'].rolling(10).mean()

In [ ]:
dfc['total_feed_rolling_std_ton_h'] = dfc['total_feed_ton_h_pv'].rolling(10).std()

In [ ]:
dfc['separator_speed_rolling_std_rpm'] = dfc['separator_speed_rpm_pv'].rolling(10).std()

In [ ]:
dfc = dfc[(dfc['total_feed_rolling_std_ton_h'] < 3) & (dfc['separator_speed_rolling_std_rpm'] < 4)]

In [ ]:
dfc = dfc[(dfc['mill_motor_pwr_kw_pv'] >= 700) & (dfc['mill_motor_pwr_kw_pv'] <= 1700)]

In [ ]:
dfc.reset_index(drop=True, inplace=True)

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(311)

plt.plot(dfc[dfc['date'] > '2019-10-31']['separator_pwr_kw_pv'])

plt.subplot(312)

plt.plot(dfc[dfc['date'] > '2019-10-31']['separator_speed_rpm_pv'])

plt.subplot(313)

plt.plot(dfc[dfc['date'] > '2019-10-31']['total_feed_ton_h_pv'])

# Mill Operation after maintaining period

In [ ]:
df['stopping_period_interval'].sort_values(ascending=False)

In [ ]:
df.loc[117268,'date']

In [ ]:
plt.figure(figsize=(20,8))

plt.plot(dfc[dfc['date'] > '2019-07-26']['mill_motor_pwr_kw_pv'])

In [ ]:
plt.figure(figsize=(20,8))

plt.plot(dfc[dfc['date'] > '2019-07-26']['mill_motor_pwr_kw_pv'].rolling(50).mean())

# Separator Motor Outliers

In [ ]:
dfc = dfc[dfc['separator_pwr_kw_pv'] < 70]

In [ ]:
plt.figure(figsize=(20,8))

plt.plot(dfc['separator_pwr_kw_pv'])

# Data preparation for Weighted Model Training

In [ ]:
all_dates = pd.DataFrame(pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='30S'), columns=['date'])

In [ ]:
all_dates_dfc = all_dates.merge(right=dfc, how='left', on='date')

In [ ]:
# len(all_dates_dfc) - all_dates_dfc['mill_motor_pwr_kw_pv'].isna().sum() 

In [ ]:
len(dfc)

In [ ]:
plt.figure(figsize=(20,8))

plt.plot(all_dates_dfc['separator_pwr_kw_pv'])

**Defining a column 'Weight' to train the model in time windows**

In [ ]:
all_dates_dfc['weight'] = all_dates_dfc['mill_operation']

In [ ]:
all_dates_dfc['weight'].replace({np.NaN : 0.0}, inplace=True)

In [ ]:
all_dates_dfc['weight'].value_counts()

In [ ]:
all_dates_dfc.drop(columns='stopping_period_interval',inplace=True)

In [ ]:
all_dates_dfc.fillna(0, inplace=True)

In [ ]:
all_dates_dfc